In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, Dense, Dropout

In [ ]:
def load_cifar10():
  (train_X, train_y), (test_X, test_y) = tf.keras.datasets.cifar10.load_data()

  train_X = train_X / 255.0
  test_X = test_X / 255.0

  train_y = to_categorical(train_y, num_classes=10)
  test_y = to_categorical(test_y, num_classes=10)
  
  return train_X, train_y, test_X, test_y

In [ ]:
def build_cnn_model(num_classes, input_shape):
  model = Sequential()

  model.add(Input(shape=input_shape))

  model.add( Conv2D(16, (3,3), padding='same', activation='relu'))
  model.add( Conv2D(16, (3,3), padding='same', activation='relu'))
  model.add(MaxPool2D())

  model.add( Conv2D(16, (3,3), padding='same', activation='relu'))
  model.add( Conv2D(16, (3,3), padding='same', activation='relu'))
  model.add(MaxPool2D())
  model.add(GlobalAveragePooling2D())

  model.add(Dense(100, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))

  return model

In [ ]:
def main(Model=None, epochs=30):
  train_X, train_y, test_X, test_y = load_cifar10()
  print('train_y[0]', train_y[0])

  model = build_cnn_model(len(train_y[0]), train_X[0].shape)

  optimizer = SGD(learning_rate=1e-2, momentum=0.9, nesterov=True)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy')

  hist = model.fit(train_X, train_y, batch_size=32, epochs=epochs, validation_split=0.2, shuffle=True)
  
  pred_y = model.predict(test_X)

  top1_accuracy = get_topk_accuracy(test_y, pred_y)
  top3_accuracy = get_topk_accuracy(test_y, pred_y, k=3)

main()


train_y[0] [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
Epoch 1/30
1250/1250 [==============================] - 61s 48ms/step - loss: 2.1229 - val_loss: 1.8745
Epoch 2/30
1250/1250 [==============================] - 59s 47ms/step - loss: 1.8431 - val_loss: 1.6746
Epoch 3/30
1250/1250 [==============================] - 59s 47ms/step - loss: 1.6694 - val_loss: 1.5231
Epoch 4/30
1250/1250 [==============================] - 60s 48ms/step - loss: 1.5537 - val_loss: 1.5092
Epoch 5/30
1250/1250 [==============================] - 60s 48ms/step - loss: 1.4614 - val_loss: 1.3315
Epoch 6/30
1250/1250 [==============================] - 60s 48ms/step - loss: 1.3868 - val_loss: 1.3136
Epoch 7/30
1250/1250 [==============================] - 59s 47ms/step - loss: 1.3299 - val_loss: 1.2811
Epoch 8/30
1250/1250 [==============================] - 59s 47ms/step - loss: 1.2917 - val_loss: 1.1948
Epoch 9/30
1250/1250 [==============================] - 58s 46ms/step - loss: 1.2479 - val_loss: 1.1473
Epoch 10/30
1250/1250

In [ ]:
def get_topk_accuracy(test_y, pred_y, k=1):
  true_labels = np.argmax(test_y, axis=1)
  pred_labels = np.argsort(pred_y, axis=1)
  correct = 0
  for true_label, pred_label in zip(true_labels, pred_labels):
    cur_preds = pred_labels[-k:]

    if true_label in cur_preds:
      correct += 1